<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Grid Search로 KNN 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [40]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [41]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/titanic.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [42]:
# 상위 몇 개 행 확인
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [43]:
# 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [44]:
# 기초통계량 확인
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [45]:
# NaN 값 확인
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) Title 변수 추가**

In [46]:
# 호칭 추출
data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [47]:
# 대표 호칭 이외는 Others로 변경
main_tit_list = ['Mr', 'Miss', 'Mrs']
data.loc[data['Title'].isin(main_tit_list)==False, ['Title']] = 'Others'

**2) 결측치 처리**

In [48]:
# Age 중앙값으로 채우기
data['Age'].fillna(data.groupby('Title')['Age'].transform('median'), inplace=True)

In [49]:
# Embarked 최빈값으로 채우기
data['Embarked'] = data['Embarked'].fillna('S')

**3) AgeGroup 변수 추가**

In [50]:
# 변수 추가
bins = [0, 16, 25, 35, 60, 100]
data['AgeGroup'] = pd.cut(data['Age'], 
                          bins=bins, 
                          labels=['0_16', '17_25', '26_35', '36_60', '61_100'])

**4) 가변수화**

In [51]:
# 가변수화
dummy_vars = ['Title', 'AgeGroup', 'Pclass', 'Sex', 'Embarked']

for v in dummy_vars:
    dummies = pd.get_dummies(data[v], prefix=v, drop_first=True)
    data = pd.concat([data, dummies], axis=1)
    data.drop(v, axis=1, inplace=True)

**5) 불필요한 변수 제거**

In [52]:
# 불필요한 열 제거
drop_cols = ['PassengerId', 'Name', 'Age', 'Ticket', 'Cabin']
data.drop(drop_cols, axis=1, inplace=True)

**6) x, y 분리**

In [53]:
# target 확인
target = 'Survived'

# 데이터 분리
x = data.drop(target, axis = 1)
y = data[target] 

**7) 정규화**

In [54]:
# 정규화
cols = ['SibSp', 'Parch', 'Fare']
x[cols] = (x[cols] - x[cols].min()) / (x[cols].max() - x[cols].min())

# 확인
x.head()

,SibSp,Parch,Fare,Title_Mr,Title_Mrs,Title_Others,AgeGroup_17_25,AgeGroup_26_35,AgeGroup_36_60,AgeGroup_61_100,Pclass_2,Pclass_3,Sex_male,Embarked_Q,Embarked_S
0,0.125,0.0,0.014151,1,0,0,1,0,0,0,0,1,1,0,1
1,0.125,0.0,0.139136,0,1,0,0,0,1,0,0,0,0,0,0
2,0.000,0.0,0.015469,0,0,0,0,1,0,0,0,1,0,0,1
3,0.125,0.0,0.103644,0,1,0,0,1,0,0,0,0,0,0,1
4,0.000,0.0,0.015713,1,0,0,0,1,0,0,0,1,1,0,1


**8) 학습용, 평가용 데이터 분리**

In [55]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 모델 선언시 n_neighbors=5를 지정합니다.

In [56]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [57]:
# 선언하기
model = KNeighborsClassifier(n_neighbors=5) # n_neighbors=5 : 디폴트 값

In [58]:
# 학습하기
model.fit(x_train, y_train)

KNeighborsClassifier()

In [59]:
# 예측하기
y_pred = model.predict(x_test)

In [60]:
# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[152  16]
 [ 34  66]]
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       168
           1       0.80      0.66      0.73       100

    accuracy                           0.81       268
   macro avg       0.81      0.78      0.79       268
weighted avg       0.81      0.81      0.81       268



# 5.성능 튜닝

- Grid Search로 튜닝을 진행합니다. 

**1) 모델링 & 튜닝**

In [61]:
# 파라미터 선언
  # n_neighbors: range(5, 31)
grid_param = {'n_neighbors':range(5, 31)}

# 확인
grid_param

{'n_neighbors': range(5, 31)}

In [62]:
# 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [63]:
# 선언하기
model_knn = KNeighborsClassifier()

# Grid Search 모델 선언하기
  # cv=10
  # scoring='accuracy'
model = GridSearchCV(model_knn, # 기본 모델 이름
                    grid_param, # 앞에서 선언한 튜닝용 파라미터 변수
                    cv=10,  # K-Fold Cross Validation(default=5)
                    scoring='accuracy',  # 평가 방법
                    n_jobs=-1) # cpu성능 좋아지게 병렬처리

In [64]:
# 학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(5, 31)}, scoring='accuracy')

**2) 결과 확인**

In [65]:
model.cv_results_ 

{'mean_fit_time': array([0.01078861, 0.00849185, 0.00820777, 0.00887523, 0.00853856,
        0.00859749, 0.00859804, 0.00940607, 0.009235  , 0.00770612,
        0.0085629 , 0.01175747, 0.00847828, 0.00841146, 0.00842161,
        0.00892026, 0.01179819, 0.01168566, 0.00883083, 0.00955045,
        0.01096466, 0.01245515, 0.00793412, 0.00963507, 0.01073704,
        0.00780141]),
 'std_fit_time': array([0.00249811, 0.00087067, 0.00155461, 0.00146013, 0.00155833,
        0.00282234, 0.00109951, 0.00367293, 0.00118294, 0.00103042,
        0.00191554, 0.00408546, 0.0008226 , 0.00085852, 0.00128033,
        0.0013489 , 0.00389235, 0.00246141, 0.001252  , 0.00112355,
        0.0031832 , 0.00853314, 0.00141415, 0.00332345, 0.00648872,
        0.00079988]),
 'mean_score_time': array([0.01127021, 0.01100519, 0.01093616, 0.01191049, 0.01331527,
        0.01005054, 0.01247561, 0.01073992, 0.01143165, 0.01178012,
        0.01176012, 0.01572475, 0.01014512, 0.0113951 , 0.0120213 ,
        0.01381364, 

In [66]:
# mean_test_score 확인
model.cv_results_['mean_test_score']

array([0.83806964, 0.83486943, 0.82680492, 0.82043011, 0.8172043 ,
       0.8156426 , 0.81075269, 0.80919099, 0.80273938, 0.80755248,
       0.79631336, 0.80271377, 0.79953917, 0.79951357, 0.79470046,
       0.79795187, 0.79951357, 0.80593958, 0.80593958, 0.80268817,
       0.80911418, 0.80755248, 0.81236559, 0.81387609, 0.81231439,
       0.81233999])

In [67]:
# 최적 파라미터
model.best_params_

{'n_neighbors': 5}

In [68]:
# 최고 성능
model.best_score_

0.8380696364567332

In [69]:
# 최적의 성능을 보이는 모델 확인
model.best_estimator_.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [70]:
# 예측하기
y_pred = model.predict(x_test)

In [71]:
# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[152  16]
 [ 34  66]]
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       168
           1       0.80      0.66      0.73       100

    accuracy                           0.81       268
   macro avg       0.81      0.78      0.79       268
weighted avg       0.81      0.81      0.81       268

